In [9]:
import re

This wonderful thread fixed 2+ hours of mucking around....

http://stackoverflow.com/questions/375427/non-blocking-read-on-a-subprocess-pipe-in-python

In [10]:
import sys
from subprocess import PIPE, Popen
from threading  import Thread

try:
    from Queue import Queue, Empty
except ImportError:
    from queue import Queue, Empty  # python 3.x

ON_POSIX = 'posix' in sys.builtin_module_names

def enqueue_output(out, queue):
    for line in iter(out.readline, b''):
        queue.put(line)
    out.close()

pianobar = Popen(['pianobar'], stdin=PIPE, stdout=PIPE, bufsize=1, close_fds=ON_POSIX)
q = Queue()
t = Thread(target=enqueue_output, args=(pianobar.stdout, q))
t.daemon = True # thread dies with the program
t.start()

# ... do other things here

# read line without blocking
#try:  line = q.get_nowait() # or q.get(timeout=.1)
#except Empty:
#    print('no output yet')
#else: # got line
#    # ... do something with line

In [ ]:
try:  line = q.get_nowait() # or q.get(timeout=.1)
    except Empty:
        print('Reached the end of current output')
        

In [11]:
test = r'	10) q   Cream Radio'
p = re.compile('(\d{1,})\)[\sqS]{5}(.*)')
m = p.search(test)
(i, n) = m.groups()
print(i)
print(n)

10
Cream Radio


In [12]:
channels = []
while True:
    try:  line = q.get_nowait() # or q.get(timeout=.1)
    except Empty:
        print('Reached the end of current output')
        break
    else: # got line
        test = line.rstrip().decode("utf-8")
        m = p.search(test)
        print(test)
        if m is not None:
            (index, name) = p.search(test).groups()
            print("found: {} {}".format(name, index))
            channels.append(name)
            
print(channels)

Welcome to pianobar (2013.09.15)! Press ? for a list of commands.
(i) Login... Ok.
(i) Get stations... Ok.
	 0) q   Abba Radio
found: Abba Radio 0
	 1) q   Adele Radio
found: Adele Radio 1
	 2) q   Aerosmith Radio
found: Aerosmith Radio 2
	 3) q   Alternative Pop/Rock
found: Alternative Pop/Rock 3
	 4) q   Billy Joel Radio
found: Billy Joel Radio 4
	 5) q   Bond Radio
found: Bond Radio 5
	 6) q   Cake Radio
found: Cake Radio 6
	 7) q   Carry On Wayward Son Radio
found: Carry On Wayward Son Radio 7
	 8) q   Classic Rock
found: Classic Rock 8
	 9) q   Classical Guitar Radio
found: Classical Guitar Radio 9
	10) q   Cream Radio
found: Cream Radio 10
	11) q S Disney (Children's) Radio
found: Disney (Children's) Radio 11
	12) q S Easy Listening
found: Easy Listening 12
	13) q   Faunts Radio
found: Faunts Radio 13
	14) q   Firework Radio
found: Firework Radio 14
	15) q   Flogging Molly Radio
found: Flogging Molly Radio 15
	16) q S Folk / Country Rock
found: Folk / Country Rock 16
	17) q   Fro

In [13]:
channel_index = 0
level = 'menu'

In [20]:
def start_channel(channel_index):
    cmd = bytearray('s'+repr(channel_index)+'\n', 'utf-8')
    print(cmd)
    pianobar.stdin.write(cmd)
    pianobar.stdin.flush()

In [21]:
start_channel(0)

bytearray(b's0\n')
